<a href="https://colab.research.google.com/github/riacheruvu/automated-data-science/blob/master/Automated_Dataset_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Dataset Selection

This is a preliminary, work in progress attempt to parse results from Google Data Search using BeautifulSoup in response to a query ("business problem") and apply fuzzy string matching to the query in order to determine the appropriate dataset.

Please ignore the unclean code and comments; a thorough clean-up and revision of the implementation is coming soon.

# Import Necessary Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import webbrowser
import requests
#from urllib.request import urlopen as uReq
import urllib
!pip install fuzzywuzzy
from bs4 import BeautifulSoup
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from ipywidgets import widgets
from IPython.display import display

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Read and analyze your request

Hi! This tool is meant for supervised learning applications (in other words, given some data, the machine learning algorithm will make predictions). It also only supports prediction of a single feature at the moment.

Answer a few questions about the type of business problem you're interested in, and we'll whip up some datasets you might be interested in.

In [3]:
#Code for Ipython widgets

from ipywidgets import widgets
from IPython.display import display
text = widgets.Text()
new_text = widgets.Text()
even_newer_text = widgets.Text()

print("What is the topic of the business problem *in a few words*?")
display(text)

print("What do you want to predict (your target variable)?")
display(new_text)

print("What information do you want to use to predict the target variable?")
display(even_newer_text)

def handle_topic_submit(sender):
    return text.value

def handle_submit(sender):
    print("Got it!", sender.value)
    
text.on_submit(handle_topic_submit)
new_text.on_submit(handle_submit)
variables = new_text.value
even_newer_text.on_submit(handle_submit)


What is the topic of the business problem *in a few words*?


Text(value='')

What do you want to predict (your target variable)?


Text(value='')

What information do you want to use to predict the target variable?


Text(value='')

In [27]:
#type_of_dataset = text.value
type_of_dataset = input("What is the topic of the business problem would you like to solve? For example, try 'Boston Education Data'\n")

stop_words = set(stopwords.words('english')) 
  
word_tokens = word_tokenize(type_of_dataset) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 

What is the topic of the business problem would you like to solve? For example, try 'Boston Education Data'
Boston Education Data


In [28]:
print(filtered_sentence)

['Boston', 'Education', 'Data']


In [0]:
#Unused code
string = [x.strip() for x in variables.split(',')]
predicted_variable = string[0]
feature = string[1]

IndexError: ignored

# Find and download the right dataset for you

In [0]:
URL = 'https://toolbox.google.com/datasetsearch/search?query=' + filtered_sentence[0] + '%20' + filtered_sentence[1] + '%20' + filtered_sentence[2]

In [0]:
data = requests.get(URL) #gets all the content of webpage
soup = BeautifulSoup(data.text,'html.parser')

In [0]:
hi = soup.prettify()
hi = soup.get_text()

In [0]:
hi = str(hi)
#unwanted = soup.find('div', {'class' : 'iKH1Bc'})
hi = hi.split("data:function(){return ",1)[1]
l = hi.replace('null','')
p = l.replace('\n', '')

In [34]:
import re
#Get Next Name of Dataset
compare = []

rest = p.split("'\\',", 1)[0]
#txtl = rest.split('"https://www.')[1::2]
#txt2 = str(txtl).split(',')[0]
rest

'["Boston Education Data",[[,,[,["BOSTON EDUCATION SKILLS \\u0026 TRAINING CORP, fiscal year ending June 2016"],["https://projects.propublica.org/nonprofits/organizations/200917822","projects.propublica.org",,,"propublica.org"],,,,,,,,,,,,,,,,,,,,,,,[["ProPublica","https://www.propublica.org/","https://t3.gstatic.com/images?q\\u003dtbn:ANd9GcQ1dChJXB5pbJLxU2cnp_0VWla6p3oHF5Cms-6QRTtG4qjRYMoL"]],,[[,"\\u003cp class\\u003d\\"boqResearchsciencesearchdesktopuiDetailDescriptionElement_\\"\\u003eProPublica\'s Nonprofit Explorer lets you view summaries of 2.2 million tax returns from tax-exempt organizations and see financial details such as their executive compensation and revenue and expenses. You can browse raw IRS data released since 2013 and access over 9.4 million tax filing documents going back as far as 2001.\\u003c/p\\u003e"]]],"4e2LlIyFc6zJo0rtAAAAAA\\u003d\\u003d"],[,,[,["CENTER FOR COLLABORATIVE EDUCATION METRO BOSTON INC, fiscal year ending June 2016"],["https://projects.propubli

In [35]:
#rest = re.findall('"([^"]*)"', rest)

rest = re.findall('"([^"]*)"', rest)
#print(rest[8])
#compare.append(rest[8])
#data_url = rest[2]
rest
#print(data_url)

['Boston Education Data',
 'BOSTON EDUCATION SKILLS \\u0026 TRAINING CORP, fiscal year ending June 2016',
 'https://projects.propublica.org/nonprofits/organizations/200917822',
 'projects.propublica.org',
 'propublica.org',
 'ProPublica',
 'https://www.propublica.org/',
 'https://t3.gstatic.com/images?q\\u003dtbn:ANd9GcQ1dChJXB5pbJLxU2cnp_0VWla6p3oHF5Cms-6QRTtG4qjRYMoL',
 '\\u003cp class\\u003d\\',
 "\\u003eProPublica's Nonprofit Explorer lets you view summaries of 2.2 million tax returns from tax-exempt organizations and see financial details such as their executive compensation and revenue and expenses. You can browse raw IRS data released since 2013 and access over 9.4 million tax filing documents going back as far as 2001.\\u003c/p\\u003e",
 '4e2LlIyFc6zJo0rtAAAAAA\\u003d\\u003d',
 'CENTER FOR COLLABORATIVE EDUCATION METRO BOSTON INC, fiscal year ending June 2016',
 'https://projects.propublica.org/nonprofits/organizations/43241676',
 'projects.propublica.org',
 'propublica.org',
 

In [0]:
#Code reused from https://stackoverflow.com/questions/28050350/split-list-of-strings-into-list-of-sublists-based-on-a-string
questionchunks = []
hithere = []
qlist = []
#rest.append('\\') # append an empty str at the end to avoid the other condn
for line in rest:

    if (line != '\\' ):
        questionchunks.append(line)      # add the element to each of your chunk   
    else: 
        qlist.append(questionchunks)   # append chunk
        questionchunks = []       # reset chunk

In [37]:
flat_list = [item for sublist in qlist for item in sublist]
#import numpy
#a = np.array(qlist)
#b = a.flatten()
#b
#a = a.tolist()
#a
flat_list

['Boston Education Data',
 'BOSTON EDUCATION SKILLS \\u0026 TRAINING CORP, fiscal year ending June 2016',
 'https://projects.propublica.org/nonprofits/organizations/200917822',
 'projects.propublica.org',
 'propublica.org',
 'ProPublica',
 'https://www.propublica.org/',
 'https://t3.gstatic.com/images?q\\u003dtbn:ANd9GcQ1dChJXB5pbJLxU2cnp_0VWla6p3oHF5Cms-6QRTtG4qjRYMoL',
 '\\u003cp class\\u003d\\',
 "\\u003eProPublica's Nonprofit Explorer lets you view summaries of 2.2 million tax returns from tax-exempt organizations and see financial details such as their executive compensation and revenue and expenses. You can browse raw IRS data released since 2013 and access over 9.4 million tax filing documents going back as far as 2001.\\u003c/p\\u003e",
 '4e2LlIyFc6zJo0rtAAAAAA\\u003d\\u003d',
 'CENTER FOR COLLABORATIVE EDUCATION METRO BOSTON INC, fiscal year ending June 2016',
 'https://projects.propublica.org/nonprofits/organizations/43241676',
 'projects.propublica.org',
 'propublica.org',
 

In [38]:
real_list=[]
real_list.append([item.split() for item in ' '.join(flat_list).split('AAAAAA') if item])
#real_list[0][1]
c= 0
for x in real_list:
    for y in x:
        c = c +1
print(c)

real_list[0][0]

99


['Boston',
 'Education',
 'Data',
 'BOSTON',
 'EDUCATION',
 'SKILLS',
 '\\u0026',
 'TRAINING',
 'CORP,',
 'fiscal',
 'year',
 'ending',
 'June',
 '2016',
 'https://projects.propublica.org/nonprofits/organizations/200917822',
 'projects.propublica.org',
 'propublica.org',
 'ProPublica',
 'https://www.propublica.org/',
 'https://t3.gstatic.com/images?q\\u003dtbn:ANd9GcQ1dChJXB5pbJLxU2cnp_0VWla6p3oHF5Cms-6QRTtG4qjRYMoL',
 '\\u003cp',
 'class\\u003d\\',
 "\\u003eProPublica's",
 'Nonprofit',
 'Explorer',
 'lets',
 'you',
 'view',
 'summaries',
 'of',
 '2.2',
 'million',
 'tax',
 'returns',
 'from',
 'tax-exempt',
 'organizations',
 'and',
 'see',
 'financial',
 'details',
 'such',
 'as',
 'their',
 'executive',
 'compensation',
 'and',
 'revenue',
 'and',
 'expenses.',
 'You',
 'can',
 'browse',
 'raw',
 'IRS',
 'data',
 'released',
 'since',
 '2013',
 'and',
 'access',
 'over',
 '9.4',
 'million',
 'tax',
 'filing',
 'documents',
 'going',
 'back',
 'as',
 'far',
 'as',
 '2001.\\u003c/p\\u

In [39]:
for pi in range(0,1):
    result = (real_list[0][0])
    #print(qlist[1], "\n")
    #result = (qlist[0][24:])
    print(result, "\n")
    hello = []

    preprocessed_text = re.sub(r'.*\u003e', '\u003e', str(result))
    #line = re.sub("", '', time)
    #pine = re.sub("", '', line)
    #fine = re.sub("", '', pine)
    #fine.replace("\\", "")
    
    hello.append(preprocessed_text)

    hi = str(hello).replace("\\", "")
    bye = hi.replace("/br", " ")
    bye.replace("/p", " ")
    rep = {"src": "", "alt": 
           "", "a href": "", "pnnp": "", "class": "", "/a": "", "/em": "", "/p": "", "img": "", 
           "boqResearchsciencesearchdesktopuiDetailDescriptionElement_": "",  "blockquote": "", "u003e": "", "u003d" : "", "u003c": "", "\\": ""} 
    # define desired replacements here

    # use these three lines to do the replacement
    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], bye)
    best = re.sub('07pwN8NZpOr9gbTfAAAAAA', '', text) 
    #email = re.compile('a href')
    #email.findall(fine)
    #hifine = email.sub('', fine)
    print(best, "\n")

['Boston', 'Education', 'Data', 'BOSTON', 'EDUCATION', 'SKILLS', '\\u0026', 'TRAINING', 'CORP,', 'fiscal', 'year', 'ending', 'June', '2016', 'https://projects.propublica.org/nonprofits/organizations/200917822', 'projects.propublica.org', 'propublica.org', 'ProPublica', 'https://www.propublica.org/', 'https://t3.gstatic.com/images?q\\u003dtbn:ANd9GcQ1dChJXB5pbJLxU2cnp_0VWla6p3oHF5Cms-6QRTtG4qjRYMoL', '\\u003cp', 'class\\u003d\\', "\\u003eProPublica's", 'Nonprofit', 'Explorer', 'lets', 'you', 'view', 'summaries', 'of', '2.2', 'million', 'tax', 'returns', 'from', 'tax-exempt', 'organizations', 'and', 'see', 'financial', 'details', 'such', 'as', 'their', 'executive', 'compensation', 'and', 'revenue', 'and', 'expenses.', 'You', 'can', 'browse', 'raw', 'IRS', 'data', 'released', 'since', '2013', 'and', 'access', 'over', '9.4', 'million', 'tax', 'filing', 'documents', 'going', 'back', 'as', 'far', 'as', '2001.\\u003c/p\\u003e', '4e2LlIyFc6zJo0rt'] 

['['Boston', 'Education', 'Data', 'BOSTON',

In [0]:
new_list = str(text).replace('" ' '",', "")
newer_list = str(new_list).replace('\'', "")
another_list = str(newer_list).replace(',', "")
another_new_list = str(another_list).replace('"', "")

string = another_new_list.replace('[','').replace(']','')
text = re.sub(r'@\S+', 'Someone', string)
text = text.replace("title", " ")

In [41]:
pattern = re.compile(r"\d*([^\d\W]+)\d*")
text = re.sub(r'http\S+', '', text)
text = pattern.sub(r"\1", text)
text = text.replace("  ", " ")
text = text.rsplit(' ', 1)[0]

text

'Boston Education Data BOSTON EDUCATION SKILLS u TRAINING CORP fiscal year ending June 2016 projects.propublica.org propublica.org ProPublica  p ProPublicas Nonprofit Explorer lets you view summaries of 2.2 million tax returns from tax-exempt organizations and see financial details such as their executive compensation and revenue and expenses. You can browse raw IRS data released since 2013 and access over 9.4 million tax filing documents going back as far as 2001.'

In [42]:
x = [str(i) for i in text.split()]
x

['Boston',
 'Education',
 'Data',
 'BOSTON',
 'EDUCATION',
 'SKILLS',
 'u',
 'TRAINING',
 'CORP',
 'fiscal',
 'year',
 'ending',
 'June',
 '2016',
 'projects.propublica.org',
 'propublica.org',
 'ProPublica',
 'p',
 'ProPublicas',
 'Nonprofit',
 'Explorer',
 'lets',
 'you',
 'view',
 'summaries',
 'of',
 '2.2',
 'million',
 'tax',
 'returns',
 'from',
 'tax-exempt',
 'organizations',
 'and',
 'see',
 'financial',
 'details',
 'such',
 'as',
 'their',
 'executive',
 'compensation',
 'and',
 'revenue',
 'and',
 'expenses.',
 'You',
 'can',
 'browse',
 'raw',
 'IRS',
 'data',
 'released',
 'since',
 '2013',
 'and',
 'access',
 'over',
 '9.4',
 'million',
 'tax',
 'filing',
 'documents',
 'going',
 'back',
 'as',
 'far',
 'as',
 '2001.']

In [0]:
for word in x:
    if word.endswith(".txt") == True:
        text = text.replace(word, " ")
    if word.endswith(".csv") == True:
        text = text.replace(word, " ")
    if word.endswith(".py") == True:
        text = text.replace(word, " ")
    if word.endswith(".com") == True:
        text = text.replace(word, " ")
    if word.endswith(".org") == True:
        text = text.replace(word, " ")
    if word.endswith(".edu") == True:
        text = text.replace(word, " ")

**The final text parsed from one of the results returned by Google Data Search:  **

In [44]:
text

'Boston Education Data BOSTON EDUCATION SKILLS u TRAINING CORP fiscal year ending June 2016     ProPublica  p ProPublicas Nonprofit Explorer lets you view summaries of 2.2 million tax returns from tax-exempt organizations and see financial details such as their executive compensation and revenue and expenses. You can browse raw IRS data released since 2013 and access over 9.4 million tax filing documents going back as far as 2001.'

**Fuzzy text string matching algorithm experiments**

In [47]:
from fuzzywuzzy import fuzz
fuzz.ratio(text, 'Data from boston public schools')

7

In [48]:
from fuzzywuzzy import fuzz
fuzz.partial_ratio(text, 'Data from boston public schools')

32

In [50]:
#Best option, so far
from fuzzywuzzy import fuzz
fuzz.token_sort_ratio(text, 'Data from boston public schools')

12

In [51]:
fuzz.token_set_ratio(text, 'Data from boston public schools')

68